In [1]:
# avoid warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
#imports
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import convert_to_tensor
from sklearn.metrics.pairwise import cosine_similarity
import IPython.display as display
from PIL import Image
import matplotlib.pyplot as plt
import math
import time
from google.cloud import bigquery

In [3]:
#params
GCP_PROJECT = os.environ.get('GCP_PROJECT')
BQ_DATASET = os.environ.get('BQ_DATASET')
BUCKET_NAME = os.environ.get('BUCKET_NAME')
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

In [4]:
# get table name
full_table_name = f'{GCP_PROJECT}.{BQ_DATASET}.image_data'
# write query
query = f'''
        SELECT 
            numeric_index,
            artwork_id,
            title,
            category,
            medium,
            date,
            height_cm,
            width_cm,
            image_url_template,
            collecting_institution,
            image_url_normalized
        FROM {full_table_name}
        '''

In [5]:
# get image table from big query
# instantiate client
client = bigquery.Client(project=GCP_PROJECT)
# set up query job
query_job = client.query(query)
# run query
result = query_job.result()
# store results in df
artworks_df = result.to_dataframe()

In [7]:
#create function to return relevant image data in a library
sim_indexes = [0, 4, 67, 9943, 6002]

#artworks_df.loc[2100,:]
artworks_df[artworks_df['category'] == 'Painting']
#,'category','medium','date','collecting_institution'

,numeric_index,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
3352,5059,515b5ec21b12b0e66800024a,Portrait of a Man and Boy,Painting,True,None,113.0,94.3,https://d32dm0rphc51dk.cloudfront.net/x_QmjPP6...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/x_QmjPP6...
3473,6323,515b8c0b1b12b0244a000d74,Giovanni Borgherini and His Tutor,Painting,True,None,47.0,60.6,https://d32dm0rphc51dk.cloudfront.net/mrMMfqyS...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/mrMMfqyS...
3503,23644,52165a468b3b818e07000326,Flower Still Life,Painting,True,None,75.0,58.5,https://d32dm0rphc51dk.cloudfront.net/6jmZ3p66...,"Hallwyl Museum, Stockholm",https://d32dm0rphc51dk.cloudfront.net/6jmZ3p66...
3719,2926,515b3020223afa9b3e0003f5,Woman Viewed from Behind,Painting,True,None,81.3,75.6,https://d32dm0rphc51dk.cloudfront.net/w37WPyxz...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/w37WPyxz...
3884,22859,516df3e9b31e2bd65e000aee,Head of an Old Man,Painting,True,None,48.3,36.4,https://d32dm0rphc51dk.cloudfront.net/qE7QmlkE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/qE7QmlkE...
...,...,...,...,...,...,...,...,...,...,...,...
23805,44,4eb1af0c2135c200010027b5,Figures in a Landscape,Painting,True,Joseon Dynasty (1392-1910); 17th century,24.1,20.3,https://d32dm0rphc51dk.cloudfront.net/KmdD_uGw...,None,https://d32dm0rphc51dk.cloudfront.net/KmdD_uGw...
23807,15064,515d2203769659e1fb003096,A Girl with a Broom,Painting,True,probably begun 1646/1648 and completed 1651,107.3,91.4,https://d32dm0rphc51dk.cloudfront.net/98RUxYji...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/98RUxYji...
23808,8987,515bc1b6cd4b8ed0b9001c52,The Fall of Phaeton,Painting,True,ca. 1604/1605-probably reworked c. 1606/1608,98.4,131.1,https://d32dm0rphc51dk.cloudfront.net/1G9CY_Tu...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/1G9CY_Tu...
23809,23484,51f17a00275b245f810004b9,Spring Festival on the River (also called Alon...,Painting,True,18th century copy after 12th century original,35.6,1152.8,https://d32dm0rphc51dk.cloudfront.net/pL5fY8il...,"National Palace Museum, Taipei",https://d32dm0rphc51dk.cloudfront.net/pL5fY8il...


In [17]:
paintings_df.iloc[1:200:,:]

,numeric_index,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
3473,6323,515b8c0b1b12b0244a000d74,Giovanni Borgherini and His Tutor,Painting,True,None,47.0,60.6,https://d32dm0rphc51dk.cloudfront.net/mrMMfqyS...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/mrMMfqyS...
3503,23644,52165a468b3b818e07000326,Flower Still Life,Painting,True,None,75.0,58.5,https://d32dm0rphc51dk.cloudfront.net/6jmZ3p66...,"Hallwyl Museum, Stockholm",https://d32dm0rphc51dk.cloudfront.net/6jmZ3p66...
3719,2926,515b3020223afa9b3e0003f5,Woman Viewed from Behind,Painting,True,None,81.3,75.6,https://d32dm0rphc51dk.cloudfront.net/w37WPyxz...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/w37WPyxz...
3884,22859,516df3e9b31e2bd65e000aee,Head of an Old Man,Painting,True,None,48.3,36.4,https://d32dm0rphc51dk.cloudfront.net/qE7QmlkE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/qE7QmlkE...
4386,14084,515d11685eeb1c904c00277c,Lamentation,Painting,True,None,64.9,65.7,https://d32dm0rphc51dk.cloudfront.net/otvvR8iA...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/otvvR8iA...
...,...,...,...,...,...,...,...,...,...,...,...
8816,10892,515cdbe7b5907b33b1000ca4,Lady Elizabeth Hamilton,Painting,True,1758,117.0,84.0,https://d32dm0rphc51dk.cloudfront.net/HTdIChzD...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/HTdIChzD...
8872,23657,521d1a0ac9dc2454600001d4,"Parnassus, ceiling fresco in the Villa Albani",Painting,True,1761,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/gMoG77-U...,"Villa Albani, Rome",https://d32dm0rphc51dk.cloudfront.net/gMoG77-U...
8891,11639,515ce8987b70570a130014d1,Lake Albano,Painting,True,1762,121.9,170.3,https://d32dm0rphc51dk.cloudfront.net/29rHOTBN...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/29rHOTBN...
8893,19632,516ca72bb83d23c471000958,Wealth and Benefits of the Spanish Monarchy un...,Painting,True,1762,181.8,104.3,https://d32dm0rphc51dk.cloudfront.net/RkFUQXrd...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/RkFUQXrd...


In [13]:
artworks_df[artworks_df['numeric_index'] == 8337]

,numeric_index,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
20055,8337,515bb69294714c2e380015bd,Vase of Flowers,Painting,True,1900/1903,101.0,81.9,https://d32dm0rphc51dk.cloudfront.net/7j77JUpO...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/7j77JUpO...


In [21]:
artworks_df['category'].value_counts()

category
Print                                      15844
Drawing, Collage or other Work on Paper     4281
Painting                                    2052
Name: count, dtype: int64

In [25]:
#4042 4461 3692 3639 4831
artworks_df.iloc[4831,:]

numeric_index                                                         23078
artwork_id                                         516df7dfed1712e96f000ce3
title                                                         Head of a Man
category                            Drawing, Collage or other Work on Paper
medium                                                                 True
date                                                                   None
height_cm                                                               8.4
width_cm                                                                6.7
image_url_template        https://d32dm0rphc51dk.cloudfront.net/bIH8DUj5...
collecting_institution             National Gallery of Art, Washington D.C.
image_url_normalized      https://d32dm0rphc51dk.cloudfront.net/bIH8DUj5...
Name: 4831, dtype: object

In [28]:
artworks_df[artworks_df['category'] == 'Print']

,numeric_index,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
0,523,515b04949562c8fee2000123,Coins [plate 9],Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/2iA-lWNm...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/2iA-lWNm...
1,534,515b04d1056351dc33000289,Diana Mourning the Death of Orion,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/35V4Sd5w...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/35V4Sd5w...
2,536,515b04d69562c8fee200012a,Saint John,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/xPc3naLE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/xPc3naLE...
3,542,515b04f205635113a50002e5,14 Juillet. Illumination du Bois de Boulogne,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/yatBEhfE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/yatBEhfE...
4,543,515b04f238ad2dfc770000d4,Sepelire mortuos,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/m67TU086...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/m67TU086...
...,...,...,...,...,...,...,...,...,...,...,...
23798,1974,515b1f34056351dc33001111,The Betrothal of the Virgin,Print,True,ca. 1504-1505 (printed 1560s/1570s),29.8,21.1,https://d32dm0rphc51dk.cloudfront.net/EFVkUwUK...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/EFVkUwUK...
23803,1747,515b1b4e9562c8fee2000ccf,Venus and Cupid,Print,True,dated 1506 (probably executed c. 1509),28.7,20.2,https://d32dm0rphc51dk.cloudfront.net/6T5wwOWb...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/6T5wwOWb...
23804,11906,515cedf0769659e1fb001738,Venus and Cupid,Print,True,dated 1506 (probably executed c. 1509),28.7,19.8,https://d32dm0rphc51dk.cloudfront.net/bZXRVaL1...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/bZXRVaL1...
23806,556,515b052f38ad2d78ca000091,Old Woman,Print,True,posthumous printing after 1919 by Felsing,29.4,24.9,https://d32dm0rphc51dk.cloudfront.net/zjmRc09n...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/zjmRc09n...


In [27]:
prints_df.head(100)

,numeric_index,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
0,523,515b04949562c8fee2000123,Coins [plate 9],Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/2iA-lWNm...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/2iA-lWNm...
1,534,515b04d1056351dc33000289,Diana Mourning the Death of Orion,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/35V4Sd5w...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/35V4Sd5w...
2,536,515b04d69562c8fee200012a,Saint John,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/xPc3naLE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/xPc3naLE...
3,542,515b04f205635113a50002e5,14 Juillet. Illumination du Bois de Boulogne,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/yatBEhfE...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/yatBEhfE...
4,543,515b04f238ad2dfc770000d4,Sepelire mortuos,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/m67TU086...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/m67TU086...
...,...,...,...,...,...,...,...,...,...,...,...
95,1164,515b113c38ad2dfc7700071d,Bunch of Grapes,Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/SaoM4KbS...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/SaoM4KbS...
96,1182,515b118938ad2d78ca000694,"Two Eagles, One Eating a Small Lamb",Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/_fXG6XeK...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/_fXG6XeK...
97,1184,515b11939562c86fb80007c8,Fagot-gatherers (Les fagottiers),Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/4djfg8ET...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/4djfg8ET...
98,1185,515b1195056351dc33000984,Women of Brussels (Femmes de Bruges),Print,True,None,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/kih5p-Fc...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/kih5p-Fc...
